In [2]:

import os
os.environ['OPENAI_API_KEY'] = 'sk-local'
import warnings
warnings.filterwarnings(
    "ignore",
    message="torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.",
)



In [3]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")


[Jul 23, 18:53:13] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [4]:
import fitz  

# Paths to the PDFs

pdf_sections = {
    'Placement Chronicles 2023-24.pdf': '=== Placement Chronicles ===\n',
    'SI Chronicles 23-24 Sem I.pdf': '=== SI Chronicles ===\n'
}

full_text = ""

for path, section_title in pdf_sections.items():
    full_text += section_title
    doc = fitz.open(path)
    for page in doc:
        full_text += page.get_text()

full_document = full_text

In [5]:
RAG.index(
    collection=[full_document],
    index_name="smile_task",
    max_document_length=180,
    split_documents=True,
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Jul 23, 18:53:41] #> Note: Output directory .ragatouille/colbert/indexes/smile_task already exists


[Jul 23, 18:53:45] [0] 		 #> Encoding 390 passages..


  0%|          | 0/13 [00:00<?, ?it/s]/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 13/13 [00:14<00:00,  1.14s/it]

[Jul 23, 18:54:00] [0] 		 avg_doclen_est = 122.96410369873047 	 len(local_sample) = 390
[Jul 23, 18:54:00] [0] 		 Creating 2,048 partitions.
[Jul 23, 18:54:00] [0] 		 *Estimated* 47,956 embeddings.
[Jul 23, 18:54:00] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/smile_task/plan.json ..


used 20 iterations (3.3486s) to cluster 45559 items into 2048 clusters
[0.034, 0.034, 0.031, 0.032, 0.033, 0.036, 0.032, 0.031, 0.032, 0.034, 0.033, 0.033, 0.033, 0.033, 0.034, 0.035, 0.033, 0.032, 0.032, 0.033, 0.033, 0.035, 0.032, 0.033, 0.031, 0.032, 0.035, 0.031, 0.036, 0.037, 0.034, 0.036, 0.035, 0.031, 0.033, 0.029, 0.035, 0.034, 0.033, 0.036, 0.034, 0.033, 0.032, 0.034, 0.034, 0.031, 0.033, 0.036, 0.033, 0.033, 0.033, 0.032, 0.037, 0.034, 0.032, 0.034, 0.036, 0.036, 0.04, 0.034, 0.032, 0.035, 0.033, 0.034, 0.036, 0.036, 0.036, 0.034, 0.032, 0.033, 0.033, 0.031, 0.033, 0.034, 0.033, 0.033, 0.035, 0.034, 0.035, 0.035, 0.037, 0.037, 0.033, 0.037, 0.033, 0.033, 0.034, 0.033, 0.032, 0.038, 0.036, 0.035, 0.033, 0.035, 0.032, 0.035, 0.036, 0.031, 0.033, 0.033, 0.033, 0.035, 0.033, 0.031, 0.036, 0.032, 0.032, 0.032, 0.033, 0.029, 0.034, 0.036, 0.034, 0.033, 0.032, 0.033, 0.037, 0.036, 0.033, 0.034, 0.033, 0.032, 0.034, 0.035, 0.032, 0.04, 0.035, 0.03]


0it [00:00, ?it/s]

...

100%|██████████| 13/13 [00:14<00:00,  1.10s/it]
1it [00:14, 14.45s/it]
100%|██████████| 1/1 [00:00<00:00, 1390.68it/s]

[Jul 23, 18:54:17] #> Optimizing IVF to store map from centroids to list of pids..
[Jul 23, 18:54:17] #> Building the emb2pid mapping..
[Jul 23, 18:54:18] len(emb2pid) = 47956



100%|██████████| 2048/2048 [00:00<00:00, 160727.76it/s]

[Jul 23, 18:54:18] #> Saved optimized IVF to .ragatouille/colbert/indexes/smile_task/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/smile_task'

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI



# RAG-Fusion: Related
template = """
You are a helpful assistant that generates multiple search queries based on a single input query.

Given an input question, generate 3 related search queries. Then, add the original input query as the 4th (final) query.

Your output should only contain the 4 queries — one per line, no explanation, no numbering.

Example:
Input: What are the effects of climate change?
Output:
what is the impact of climate change on weather patterns
how does global warming affect biodiversity
what are the long-term consequences of rising global temperatures
What are the effects of climate change?

Input: {question}
Output:
""".strip()
prompt_rag_fusion = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(model_name="phi-3.5:3b",
                 openai_api_base="http://127.0.0.1:1337/v1",temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)
print(generate_queries.invoke({"question": "What is data analytics ?"}))


['what is the role of data analytics in business decision making', 'how does data analytics improve operational efficiency', 'how can data analytics be applied in predictive modeling', 'what is data analytics ?']


In [11]:
from langchain.load import dumps, loads
from langchain_core.runnables import RunnableLambda

def sequential_map(fn):
    return RunnableLambda(lambda input: [fn.invoke(x) for x in input])

def reciprocal_rank_fusion(results: list[list], k=10):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retriever = RAG.as_langchain_retriever(k=3)
retrieval_chain_rag_fusion = generate_queries | sequential_map(retriever) | reciprocal_rank_fusion




In [13]:

docs = retrieval_chain_rag_fusion.invoke({"question":"What is data analytics ?"})
for i, (doc, score) in enumerate(docs, 1):
    print(f"\n--- Document {i} ---\n")
    print(doc.page_content.strip())
    print(f"\nScore: {score}")

/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(



--- Document 1 ---

The list of companies and roles offered by them, as 
displayed by the Placement Chronicles is non-
exhaustive.  
24
46
95
27
19
5
Analytics
05
Accenture India
06
Axxela Advisory 
Services
08
LatentView 
Analytics
10
What is Data Analytics?
The collection, organisation, and analysis of data in order to make 
informed business decisions. 
Check out some training resources provided by the Placement Unit to 
get started on your preparation for these roles -
ug Placement Training Module for Analytics and Machine Learning by the 
Career Development Committee, Placement Unit - click hereg
bg Basics of Data Visualisation and Machine Learning by the Career 
Development Committee, Placement Unit - click here.

Score: 0.28181818181818186

--- Document 2 ---

The 
first question was of medium difficulty, similar to what you might find on 
LeetCode. The second question, however, posed a greater challenge as it 
was considered hard. Remember that understanding of basic courses 

/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/opt/anaconda3/envs/rag-env/lib/python3.12/sit

In [15]:
from operator import itemgetter
from langchain_openai import ChatOpenAI


# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="phi-3.5:3b",
                 openai_api_base="http://127.0.0.1:1337/v1",temperature=0)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)



In [19]:
question = "what is the CG Cutoff for InfoEdge India Limited ? " 
print("OUTPUT:\n\n",final_rag_chain.invoke({"question": question}))


/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/opt/anaconda3/envs/rag-env/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/opt/anaconda3/envs/rag-env/lib/python3.12/sit

OUTPUT:

 The CG Cutoff for InfoEdge India Limited is 6.5+. This information is derived from the context where it mentions "Branches open to - All\nCG Cutoff - 6.5+".
